In [1]:
import openrouteservice
import pandas as pd
import folium
import pickle
pd.set_option('display.max_rows', 100)

In [2]:
client = openrouteservice.Client(key='5b3ce3597851110001cf62485a0e241a183542b1b686d0d9183a50a9')

In [3]:
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

In [4]:
df = pd.read_csv('LocationDifferentials2021.csv',header=0)
df = df.loc[df['Port']==df['NTP']]
df.set_index('Location', inplace=True)
#remove GRENFELL entry
df.drop('GRENFELL', inplace=True)

In [5]:
locations = [(f"{df.index[x]}, {df['State'].iloc[x]}", df['LD'].iloc[x]) for x in range(len(df))]
locations_set = list(set(locations))

In [6]:
len(locations_set)

456

In [7]:
#find locations with same site names but different NTP differentials
for i in range(len(df)-1):
    if (df.index[i], df['State'][i]) == (df.index[i+1], df['State'][i+1]) and (df['LD'][i] != df['LD'][i+1]):
        print(df.index[i], df['State'][i], df['LD'][i])

HAY NSW 40.75
YENDA NSW 47.0
BOORT VIC 27.75
LOCK SA 13.55


In [74]:
distance={}
demandList = []

#convert demand dictionary into list of just dict values
for value in demand.values():
    demandList.append(list(reversed(value)))
    
for key, value in coords.items():
    demandTemp = demandList.copy()
    demandTemp.insert(0,list(reversed(value)))
    distance[key] = client.distance_matrix(locations=demandTemp, profile='driving-hgv', sources=[0], destinations=list(range(1,len(demandTemp))), metrics=['distance'], units='km')

/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 5th time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openrouteservice/client.py:214: UserWarning: Rate limit exceeded. Retrying for the 6th time.
  stacklevel=1)
/opt/conda/envs/gee/lib/python3.7/site-packages/openroutes

In [75]:
#with open('./pickle/distanceMatrixHGV.pickle', 'wb') as handle:
#    pickle.dump(distance, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
#import location coordinates
with open('./pickle/coords.pickle', 'rb') as handle:
    coords = pickle.load(handle)

In [9]:
#import demand location coords
with open('./pickle/demand.pickle' ,'rb') as handle:
    demand = pickle.load(handle)

In [10]:
with open('./pickle/distanceMatrixHGV.pickle', 'rb') as handle:
    distanceMatrix = pickle.load(handle)

In [80]:
distanceSummary = {}
for sourceName, distanceValue in distanceMatrix.items():
    distanceSummary[sourceName] = {}
    distanceSummary[sourceName]['source'] = (sourceName, distanceValue['sources'][0]['location'], distanceValue['metadata']['query']['locations'][0])
    distanceSummary[sourceName]['destinations'] = []
    for demandName, demandCoord in demand.items():
        for queriedCoord in distanceValue['metadata']['query']['locations']:
            #find the matching location name for the queried coordinates
            numIndex = distanceValue['metadata']['query']['locations'].index(queriedCoord)
            destTemp = distanceValue['destinations'][numIndex-1]['location']    #numIndex-1 as the queriedCoord has 1 extra entry for the source location
            distanceTemp = distanceValue['distances'][0][numIndex-1]    #find the distance
            
            #freight rate for various km trips
            if distanceTemp < 50:
                freight = 10
            elif 50 <= distanceTemp < 100:
                freight = 15
            elif 100 <= distanceTemp < 140:
                freight = 18
            elif 140 <= distanceTemp < 180:
                freight = distanceTemp * 0.13
            elif 180 <= distanceTemp < 400:
                freight = distanceTemp * 0.12
            elif 400 <= distanceTemp < 500:
                freight = distanceTemp * 0.115
            elif 500 <= distanceTemp < 700:
                freight = distanceTemp * 0.11
            elif 700 <= distanceTemp < 800:
                freight = distanceTemp * 0.105
            elif 800 <= distanceTemp < 1000:
                freight = distanceTemp * 0.1
            elif 1000 <= distanceTemp:
                freight = distanceTemp * 0.095
            
            #add entry to distanceSummary if the queried coordinates match the demand coordinate so the correct demand name is input
            if list(reversed(demandCoord)) == queriedCoord:
                distanceSummary[sourceName]['destinations'].append([demandName, destTemp, queriedCoord, distanceTemp, round(freight,0)])

In [81]:
#with open('./pickle/distanceSummaryHGV.pickle','wb') as handle:
#    pickle.dump(distanceSummary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [12]:
with open('./pickle/distanceSummaryHGV.pickle','rb') as handle:
    distanceSummary = pickle.load(handle)

In [13]:
freightTable = {}
colNames = [x[0] for x in distanceSummary['MALU, QLD']['destinations']]
for sourceName, distanceDict in distanceSummary.items():
    freightTable[sourceName] = []
    for lists in distanceDict['destinations']:
        freightTable[sourceName].append(lists[4])

In [54]:
distanceTable = {}
colNames = [x[0] for x in distanceSummary['MALU, QLD']['destinations']]
for sourceName, distanceDict in distanceSummary.items():
    distanceTable[sourceName] = []
    for lists in distanceDict['destinations']:
        distanceTable[sourceName].append(lists[3])

In [127]:
freight_df = pd.DataFrame.from_dict(freightTable,orient='index', columns=colNames)

In [128]:
distance_df = pd.DataFrame.from_dict(distanceTable,orient='index',columns=colNames)

In [129]:
prices = pd.read_csv('./prices.csv',index_col='LOCATION').T

In [130]:
depot = -freight_df.sub(prices.loc['ASW'], axis='columns')-18

In [131]:
df_location = pd.DataFrame.from_records(locations, columns=['Location','LD'])
df_location.set_index('Location',inplace=True)

In [132]:
df_location_mod = df_location.copy()

In [133]:
df_location_mod = df_location_mod.groupby('Location').mean()

In [134]:
track = depot.add(df_location_mod['LD'], axis='index')

In [135]:
maxValues = {}
for column in track.T.columns:
    sourceMax = track.T[column].idxmax()
    MaxValue = track.T[column].max()
    maxValues[column] = (sourceMax, MaxValue, freight_df.loc[column][sourceMax], distance_df.loc[column][sourceMax])

In [136]:
m = folium.Map(location=list(reversed([117.1, -30.85])), tiles='cartodbpositron', zoom_start=13)
basemaps['Google Satellite Hybrid'].add_to(m)

In [137]:
for key,Value in maxValues.items():
    sourceCoord = coords[key]
    destinationCoord = demand[Value[0]]
    folium.PolyLine(locations=[sourceCoord,destinationCoord], color='red', weight=3, opacity=1).add_to(m)

In [138]:

for key, value in coords.items():
    popup_content = {'source': key, 'dest': maxValues[key][0], 'distance': maxValues[key][3], 'freight': maxValues[key][2]}
    html = "{source}<br>dest: {dest}<br>distance: {distance}km<br>freight: ${freight}".format(**popup_content)
    iframe = folium.IFrame(html,
                       width=250,
                       height=100)
    folium.Marker(
        location=list(value),
        icon = folium.Icon(icon='compass', color='red', prefix='fa'),
        popup=folium.Popup(iframe)).add_to(m)

In [139]:
m